In [38]:
import tensorflow as tf

# ignore warnings   
import warnings
warnings.filterwarnings('ignore')


In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

IMG_SIZE = 224
directory_data_train = "data/data4/train"
directory_data_validation = "data/data4/validation"

# Data generator for training data with augmentations
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.15,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Data generator for validation data without augmentations
val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

# Load training data
train_ds = train_datagen.flow_from_directory(
    directory=directory_data_train,
    batch_size=32,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    seed=123  
)

# Load validation data
validation_ds = val_datagen.flow_from_directory(
    directory=directory_data_validation,
    batch_size=32,
    target_size=(IMG_SIZE, IMG_SIZE),
    class_mode='categorical',
    seed=123 
)

Found 1195 images belonging to 14 classes.
Found 136 images belonging to 14 classes.


In [40]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Tải mô hình MobileNetV2 đã được huấn luyện trước mà không bao gồm lớp phân loại đầu ra
MobileNetV2_base = MobileNetV2(
    weights='imagenet', 
    include_top=False, 
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='avg')  # 'pooling='avg' sẽ tự động thêm lớp GlobalAveragePooling2D

# Đóng băng các lớp của mô hình đã được huấn luyện trước
MobileNetV2_base.trainable = False

# Xây dựng mô hình sử dụng Functional API
x = MobileNetV2_base.output

x = BatchNormalization()(x)
x = Dropout(0.3)(x)  # Giảm Dropout để giữ lại nhiều đặc trưng hơn
x = Dense(128, activation='relu')(x)
x = Dense(128, activation='relu')(x)


# Lớp đầu ra
predictions = Dense(train_ds.num_classes, activation='softmax')(x)

# Định nghĩa mô hình
model = tf.keras.Model(inputs=MobileNetV2_base.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])


In [41]:

history= model.fit(train_ds,
    validation_data=validation_ds,
    steps_per_epoch=len(train_ds),
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='accuracy',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 36s 747ms/step - accuracy: 0.4016 - loss: 2.0208 - val_accuracy: 0.9191 - val_loss: 0.4837
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9191 - val_loss: 0.4837
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 713ms/step - accuracy: 0.8793 - loss: 0.4037 - val_accuracy: 0.9485 - val_loss: 0.2423
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9485 - val_loss: 0.2423
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 725ms/step - accuracy: 0.9194 - loss: 0.2763 - val_accuracy: 0.9632 - val_loss: 0.1573
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 75ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9632 - val_loss: 0.1573
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 32s 752ms/step - accuracy: 0.9259 - loss: 0.2129 - val_accuracy: 0.9485 - val_loss: 0.1705
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.0000e+00 - los

In [42]:
# model.summary()

In [43]:
import pickle
# Lưu lịch sử huấn luyện
with open('history.pkl', 'wb') as f:
    pickle.dump(history.history, f)

In [44]:
# Lưu mô hình
model.save("model.h5")
print(tf.__version__)

2.17.0
